In [1]:
%cd /content


/content


In [2]:
!git clone https://github.com/ultralytics/yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 17646, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 17646 (delta 13), reused 8 (delta 8), pack-reused 17601 (from 3)
Receiving objects: 100% (17646/17646), 16.99 MiB | 18.39 MiB/s, done.
Resolving deltas: 100% (11989/11989), done.


In [3]:
# ================================
# ✅ Install YOLOv8 (Ultralytics)
# ================================
!pip install "ultralytics<=8.3.40" -q

from IPython import display
display.clear_output()

# Disable telemetry (privacy)
!yolo settings sync=False

import ultralytics
ultralytics.checks()
print("✅ YOLOv8 installed successfully!")


Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.2/107.7 GB disk)
✅ YOLOv8 installed successfully!


In [4]:
# ================================
# ✅ Install Detectron2 (Facebook AI)
# ================================
!pip install 'git+https://github.com/facebookresearch/detectron2.git' -q

from IPython import display
display.clear_output()

import detectron2
print("✅ Detectron2 installed successfully! Version:", detectron2.__version__)


✅ Detectron2 installed successfully! Version: 0.6


In [5]:
# ================================
# ✅ Install Supervision (with assets)
# ================================
!pip install supervision[assets] -q

from IPython import display
display.clear_output()

import supervision as sv
print("✅ Supervision installed successfully! Version:", sv.__version__)


✅ Supervision installed successfully! Version: 0.26.1


In [7]:
!pip install -U supervision


In [8]:
import supervision as sv
print("Supervision version:", sv.__version__)


Supervision version: 0.26.1


In [16]:
# =====================================================
# 🚀 YOLOv8 + Supervision Dynamic People Counter
# =====================================================
!pip install -q ultralytics supervision opencv-python

import supervision as sv
from ultralytics import YOLO
import cv2, numpy as np
from google.colab import files

# ========================
# 1️⃣ Upload your video
# ========================
print("📤 Please upload your MP4 video file...")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"✅ Uploaded: {video_path}")

# ========================
# 2️⃣ Load YOLOv8 model
# ========================
model = YOLO("yolov8n.pt")  # lightweight & fast
print("✅ YOLOv8 model loaded!")

# ========================
# 3️⃣ Annotators setup
# ========================
try:
    box_annotator = sv.BoundingBoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1)
    USE_SUPERVISION_ANNOTATORS = True
    print("✅ Using Supervision annotators")
except AttributeError:
    USE_SUPERVISION_ANNOTATORS = False
    print("⚠️ Using fallback drawing (older supervision version)")

def draw_boxes_fallback(frame, detections):
    """Manual fallback box drawing if supervision is older"""
    for xyxy, conf, cls in zip(detections.xyxy, detections.confidence, detections.class_id):
        x1, y1, x2, y2 = map(int, xyxy)
        label = f"{model.names.get(cls, 'obj')} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return frame

# ========================
# 4️⃣ Process video dynamically
# ========================
video_info = sv.VideoInfo.from_video_path(video_path)
frame_gen = sv.get_video_frames_generator(video_path)

output_path = "people_dynamic_count.mp4"
out = cv2.VideoWriter(
    output_path, cv2.VideoWriter_fourcc(*'mp4v'),
    video_info.fps, (video_info.width, video_info.height)
)

frame_count = 0
print("🎥 Processing video... Dynamic people count enabled.")

for frame in frame_gen:
    frame_count += 1

    # Run YOLOv8 detection
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Only keep people (class 0)
    detections = detections[detections.class_id == 0]

    # Live count = number of people detected in this frame
    current_people = len(detections.xyxy)

    # Annotate boxes and labels
    if USE_SUPERVISION_ANNOTATORS:
        annotated = box_annotator.annotate(scene=frame.copy(), detections=detections)
        annotated = label_annotator.annotate(scene=annotated, detections=detections)
    else:
        annotated = draw_boxes_fallback(frame.copy(), detections)

    # Display live count
    cv2.putText(
        annotated,
        f"People Now: {current_people}",
        (30, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.5,
        (0, 255, 255),
        3
    )

    out.write(annotated)

    if frame_count % 30 == 0:
        print(f"Processed {frame_count} frames... Current People={current_people}")

out.release()
print(f"✅ Done! Processed {frame_count} frames.")
print(f"🎬 Saved as {output_path}")

# ========================
# 5️⃣ Download result
# ========================
files.download(output_path)


📤 Please upload your MP4 video file...


Saving train.mp4 to train.mp4
✅ Uploaded: train.mp4
✅ YOLOv8 model loaded!
⚠️ Using fallback drawing (older supervision version)
🎥 Processing video... Dynamic people count enabled.
Processed 30 frames... Current People=0
Processed 60 frames... Current People=2
Processed 90 frames... Current People=2
Processed 120 frames... Current People=2
Processed 150 frames... Current People=2
Processed 180 frames... Current People=2
Processed 210 frames... Current People=2
Processed 240 frames... Current People=2
Processed 270 frames... Current People=2
Processed 300 frames... Current People=2
Processed 330 frames... Current People=2
Processed 360 frames... Current People=1
Processed 390 frames... Current People=1
Processed 420 frames... Current People=1
Processed 450 frames... Current People=1
✅ Done! Processed 450 frames.
🎬 Saved as people_dynamic_count.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>